In [1]:
import json
import json_coder
import pandas as pd
import networkx as nx
import befaas as bf
import befaas.logentry as le
from befaas.graph import build_function_graph, add_default_metadata, conv_to_ms

filepath = "../dumps/"
logdumps = ["dump_streaming_google.json","dump_streaming_aws.json","dump_streaming_azure.json"]
#logdumps = ["dump_streaming_google.json","dump_streaming_aws.json"]
#logdumps = ["dump_streaming_aws.json"]
#logdumps = ["dump_streaming_azure.json"]
outfile = "../img/2023_streaming_coldStarts.pkl"
plotdata = []

print("Everything is set up.")

Everything is set up.


In [2]:

debugpairs = []
for dump in logdumps:
    print(f"Include dump {dump} ...")
    print("Load dump... (may take a while)")
    data = bf.load_logs(filepath + dump)
    print("Done.")

    print("Sort data to contexts and find cold starts...")
    contexts = {}
    coldtsarts = []
    experimentStart = None

    for entry in data:
        id = entry.context_id
        if id == None:
            if isinstance(entry, le.ColdstartLog):
                coldtsarts.append(entry)
            else:
                print("Some other log entry type")

        if not id == None:
            if not (id in contexts):
                contexts[id] = []

            contexts[id].append(entry)
    print("Done.")

    print("Find xpairs...")
    xpairs = {}

    for ctx_id in contexts:
        ctx = contexts[ctx_id]
        for entry in ctx:
            id = entry.event["xPair"]
            if not (id in xpairs):
                xpairs[id] = []

            specialPair = False
            if isinstance(entry, le.PerfLog):
                if not entry.perf_type_data == "":
                    parts = entry.perf_type_data.split(":")
                    if len(parts) > 1:
                        id = parts[1]
                        if not (id in xpairs):
                            xpairs[id] = []
                        xpairs[id].append(entry)
                        specialPair = True

            if not specialPair:
                xpairs[id].append(entry)
    print("done.")

    errors = []

    print("Find experiment start...")
    for xpairID in xpairs:
        xpair = xpairs[xpairID]
        for entry in xpair:
            if not isinstance(entry, le.ArtilleryLog):
                if experimentStart is None :
                    experimentStart = entry.timestamp
                if entry.timestamp < experimentStart:
                    experimentStart = entry.timestamp
    print("done.")

    n = 0
    for xpairID in xpairs:
        xpair = xpairs[xpairID]

        requestStart = None
        requestEnd = None
        function = ""
        platform = ""
        latency = -10

        if len(xpair) == 2:
            if str(xpair[0].url).__contains__("azure"):
                continue
                for entry in xpair:
                    if isinstance(entry, le.ArtilleryLog):
                        if entry.type == "before":
                           requestStart = entry.timestamp
                        if entry.type == "after":
                           requestEnd = entry.timestamp
                           function = str(entry.url).split("/")[-1]
                           platform = "azure"
            else:
                print("Error for " + xpair[0].url)
        else:
            for entry in xpair:
                if isinstance(entry, le.PerfLog):
                    if str(entry.perf_type[0]) == "start" and not str(entry.perf_type[1]).__contains__("db"):
                        requestStart = entry.timestamp
                    if str(entry.perf_type[0]) == "measure" and not str(entry.perf_type[1]).__contains__("db"):
                        latency = entry.perf["duration"]
                if isinstance(entry, le.RequestLog):
                    function = entry.function
                    platform = entry.platform
                    if requestStart == None:
                        requestStart = entry.timestamp
                    else:
                        if (entry.timestamp < requestStart):
                            requestStart = entry.timestamp

        #latency = (requestEnd - requestStart).microseconds / 1000

        if not platform == "" and latency > 0 and requestStart is not None :
            row = {}
            row["Provider"] = platform
            row["function"] = function
            row["kind"] = "request"
            row["start"] = (requestStart - experimentStart).seconds
            row["latency"] = latency

            if row["start"] < 4000:
                plotdata.append(row)
                n = n+1
            else:
                print("Error: " + json.dumps(row))
        else:
            debugpairs.append(xpair)

    print("n=" + str(n) + " measurements for " + platform)

    print("Analyze cold starts...")

    for cold in coldtsarts:
        if not cold.platform == "artillery":
            row = {}
            row["Provider"] = cold.platform
            row["function"] = cold.function
            row["kind"] = "coldstart"
            start = (cold.timestamp - experimentStart).seconds
            if start > 80000:
                start = start - 86400
            if start < 0:
                start = 0
            row["start"] = start
            row["latency"] = 0

            if row["start"] < 2100:
                plotdata.append(row)
            else:
                print("Error: " + json.dumps(row))
                debugpairs.append(cold)


    print("Done.")

Include dump dump_streaming_google.json ...
Load dump... (may take a while)
Done.
Sort data to contexts and find cold starts...
Done.
Find xpairs...
done.
Find experiment start...
done.
n=15257 measurements for google
Analyze cold starts...
Done.
Include dump dump_streaming_aws.json ...
Load dump... (may take a while)
Done.
Sort data to contexts and find cold starts...
Done.
Find xpairs...
done.
Find experiment start...
done.
n=15454 measurements for aws
Analyze cold starts...
Error: {"Provider": "aws", "function": "patchUserMeta", "kind": "coldstart", "start": 2208, "latency": 0}
Done.
Include dump dump_streaming_azure.json ...
Load dump... (may take a while)
Done.
Sort data to contexts and find cold starts...
Done.
Find xpairs...
done.
Find experiment start...
done.
n=9143 measurements for 
Analyze cold starts...
Error: {"Provider": "azure", "function": "addVideo", "kind": "coldstart", "start": 6828, "latency": 0}
Error: {"Provider": "azure", "function": "authDevice", "kind": "coldst

In [3]:
df_calls = pd.DataFrame(plotdata)
print(df_calls.head())
df_calls.describe()

  Provider        function     kind  start    latency
0   google    registerUser  request      0  14.187894
1   google  registerDevice  request      0  87.973293
2   google  registerDevice  request      0  17.420433
3   google    registerUser  request      0  10.535028
4   google  registerDevice  request      0  17.148184


,start,latency
count,39960.000000,39960.000000
mean,1245.856957,12.135366
std,762.566179,17.117890
min,0.000000,0.000000
25%,211.000000,2.382563
50%,1682.000000,4.702271
75%,1794.000000,17.154608
max,2002.000000,327.226993


In [4]:
df_calls.to_pickle(outfile)